# FargoCPT Output Structure

This notebook introduces the structure of the simulation output directory and introduces a tool to investigate it.

We will use the data from the simulation in the quickstart example, so make sure you ran this beforehand.

## The output directory structure

The following cell will stop the notebook if the simulation has not been run yet.

In [1]:
import os
example_name = "100_quickstart"
if not os.path.basename(os.getcwd()) == example_name:
    if os.path.exists(example_name):
        os.chdir(example_name)
        if not os.path.exists("output/out/snapshots/list.txt"):
            raise FileNotFoundError(f"No snapshots found. Please run the simulation inside the 100_Quickstart.ipynb notebook first!")
    else:
        raise FileNotFoundError(f"Please go through the 100_Quickstart.ipynb notebook first!")
print(f"We are now in the directory {os.getcwd()}")

We are now in the directory /workspaces/fargocpt/examples/100_quickstart


Now, let's have a look at how the output directory is structured.

In [2]:
!ls output/out

Sigma1D.info	fargocpt_output_v1_2  snapshots     vrad1D.info
dimensions.dat	monitor		      units.dat     vtheta1D.info
energy1D.info	parameters	      used_rad.dat


It contains some files describing general properties of the simulation, like 
- the version of the code (`fargocpt_output_v1_2`),
- the dimensions of the grid (`dimensions.dat` and `used_rad.dat`),
- the code units used (`units.dat`),
- and information on 1D output files (`*1D.info`) used to load the corresponding binary files.

There is also a `parameters` directory containing a copy of the setup file used for every start of the simulation.
This way you can easily track how often and at which snapshots you restarted a long run.
Here, we only have one copy.

In [3]:
!ls output/out/parameters

setup.yml


Next, there are snaphost directories, each containing a full snapshot of the system.
Each of these directories can be used to restart the simulation or start a new one.

In [4]:
!ls output/out/snapshots

0    14  20  27  33  4	 46  52  59  65  71  78  84  90  97
1    15  21  28  34  40  47  53  6   66  72  79  85  91  98
10   16  22  29  35  41  48  54  60  67  73  8	 86  92  99
100  17  23  3	 36  42  49  55  61  68  74  80  87  93  damping
11   18  24  30  37  43  5   56  62  69  75  81  88  94  list.txt
12   19  25  31  38  44  50  57  63  7	 76  82  89  95  timeSnapshot.dat
13   2	 26  32  39  45  51  58  64  70  77  83  9   96


The `damping` directory contains a copy of the initial data which is used in the code e.g. for damping to the initial density inside of damping zones close to the boundaries. Copy this aswell, if you want to restart a simulation from a snapshot.

There is `timeSnapshot.dat` file which is a tab separated data file containing the time of the snapshot and the `list.txt` file which is simply a text file which has the number of each snapshot in a separate line. This is useful if you interact with the code using the command line. E.g.

In [5]:
!tail -n 1 output/out/snapshots/list.txt

100


is a clean way to get the number of the last snapshot.

The `snapshots` directory contains 
- the state variables of the hydro simulation (density, energy and velocities), 
- 1D output files, 
- a binary file for each planet and the `rebound.bin` for the state of the integrator (this is used for binary exact restarting)
- the `misc.bin` file which contains the state of the simulation system, e.g. the orientation of the coordinate system w.r.t. to an inertial frame and the last used CFL limited timestep,
- and a copy of the setup at the time of this snapshot.

In [6]:
!ls output/out/snapshots/0

Sigma.dat    energy.dat    planet1.bin	vrad.dat    vtheta1D.dat
Sigma1D.dat  energy1D.dat  planet2.bin	vrad1D.dat
config.yml   misc.bin	   rebound.bin	vtheta.dat


Finally, there is the `monitor` directory which contains monitor variables. These scalar variables are computed from the system state during the simulation and are written more often than the full snapshots.

In [7]:
!ls output/out/monitor

Quantities.dat	planet1.dat  planet2.dat  timeMonitor.dat  timestepLogging.dat


All files have a header that describes the colums and the units of the variables. The header can be automatically parsed.

In [8]:
!head output/out/monitor/timeMonitor.dat

# Time log for course output.
#version: 0.1
#variable: 0 | time step | 1
#variable: 1 | analysis time step | 1
#variable: 2 | physical time | 5.0225669513368811e+06 s
# One DT is 0.314000000000000001 (code) and 1577086.02271978068 (cgs).
# Syntax: coarse output step <tab> fine output step <tab> physical time (code)
0	0	0.0000000000000000e+00
0	1	3.1400000000000000e-01
0	2	6.2800000000000000e-01


- Each planet has its own file,
- disk quantities, e.g. the total mass, are stored in `Quantities.dat`,
- the output times of these fine grained monitor variables are stored in `timeMonitor.dat`, along with the corresponding snapshot number,
- and information about the CFL timestep and the ellapsed walltime can be found in `timestepLogging.dat`.

## Loading data

Let's inspect the monitor quantities that Fargo outputs.
Those are stored in the `monitor` directory within the output dir.

We'll use the `inspect_tab_file.py` tool, which helps navigating the tab separated output files.

Calling this tool with the `monitor/Quantities.dat` file, an overview of the available data is shown.

In [9]:
!python3 ../../Tools/inspect_tab_file.py output/out/monitor/Quantities.dat

Available variables:
 0   time step
 1   analysis time step
 2   physical time
 3   mass
 4   radius
 5   angular momentum
 6   total energy
 7   internal energy
 8   kinematic energy
 9   potential energy
10   radial kinetic energy
11   azimuthal kinetic energy
12   eccentricity
13   periastron
14   viscous dissipation
15   luminosity
16   pdivv
17   inner boundary mass inflow
18   inner boundary mass outflow
19   outer boundary mass inflow
20   outer boundary mass outflow
21   wave damping inner mass creation
22   wave damping inner mass removal
23   wave damping outer mass creation
24   wave damping outer mass removal
25   density floor mass creation
26   aspect ratio
27   indirect term nbody x
28   indirect term nbody y
29   indirect term disk x
30   indirect term disk y
31   frame angle
32   disk eccentricity
33   disk periastron
34   advection torque
35   viscous torque
36   gravitational torque


In [10]:
!python3 ../../Tools/inspect_tab_file.py output/out/monitor/Quantities.dat 2 3 --units kyr solMass | head

         0 kyr	  0.000349 solMass
     5e-05 kyr	  0.000349 solMass
  9.99e-05 kyr	  0.000349 solMass
   0.00015 kyr	  0.000349 solMass
    0.0002 kyr	  0.000349 solMass
   0.00025 kyr	  0.000349 solMass
    0.0003 kyr	  0.000349 solMass
   0.00035 kyr	  0.000349 solMass
    0.0004 kyr	  0.000349 solMass
   0.00045 kyr	  0.000349 solMass
